# Imports

## Python imports and setup

In [ ]:
# Native Python
import re
import socket
import colorsys
from pathlib import Path
from copy import deepcopy
from functools import partial


# Data storage & processing
import numpy as np
import pandas as pd
import xarray as xr
from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
from PyMca5.PyMca.FsmReader import parseFile

# Embeddings
import umap
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# Plotting
import panel as pn
css = '''
.bk.custom-panel {
  background: #f0f0f0;
  padding: 50px;
  border-radius: 10px;
}
.bk.divider {
  border-top: 1px solid #d9d9d9;
  height: 5px;
}
'''
pn.config.raw_css=[css]

# import hsluv
import param
import holoviews as hv
from bokeh.models import HoverTool
from bokeh.themes.theme import Theme
from holoviews.operation import decimate
from holoviews.operation.datashader import datashade, dynspread, rasterize


# To enable saving selections
from numpy import array
from holoviews import dim
from holoviews.element.selection import spatial_select

Set holoviews width (%) and backend

In [ ]:
hv.extension("bokeh", width=100)
hv.Store.set_current_backend('bokeh')
pn.extension()

Set Bokeh theme and default holoviews *Curve* options

In [ ]:
theme = Theme(
    json={
        'attrs' : {
            'Figure' : {
                'background_fill_color': '#f0f0f0',
                'border_fill_color': '#f0f0f0',
            }
        }
    }
)

hv.renderer('bokeh').theme = theme

hv.opts.defaults(
    hv.opts.Curve(tools=["hover"], line_width=.75),
)

Set the default size of the plots, depending on the computer running it

In [ ]:
size = 250

if socket.gethostname() == "Laurens-PC":
    size = 300

### New Dashboard

In [ ]:
class Opts(param.Parameterized):
    
    xlabel = param.String()
    ylabel = param.String()
    
    xlim = param.NumericTuple()
    ylim = param.NumericTuple()
    
    title = param.String()
    aspect = param.String(None, allow_None=True)

# Region selection
class DataContainer(param.Parameterized):        
    # Data import
    filename = param.Filename()
    spectral_data = param.Array()    
    band_min = param.Number()
    band_max = param.Number()
    band_change = param.Number()
    wavenumbers = param.Array()
    
    raw_data = param.Array()
    all_data = param.Array()
    sel_data = param.Array()
    all_norm_data = param.Array()
    
    do_trans_to_abs = param.Boolean(False)
    do_baseline_shift = param.Boolean(False)
    do_normalisation = param.Boolean(False)
    shifting_band = param.Integer(-1)
    shifting_range = param.Integer(100, bounds=(1, None))
    normalisation_band = param.Integer(-1)
    normalisation_range = param.Integer(5, bounds=(1, None))
    shifting_band_selector = pn.widgets.IntInput(name="Shifting band")
    normalisation_band_selector = pn.widgets.IntInput(name="Normalisation band")
    
    # Region selection    
    x_min = param.Integer(-1)
    x_max = param.Integer(-1)
    y_min = param.Integer(-1)
    y_max = param.Integer(-1)
    sel_plot_opts = Opts(xlim=(0, 10), ylim=(0, 10))
    
    # Plotting
    band_selector = pn.widgets.IntInput(name="Plotting band")
    band = param.Selector()
    band_index = None
    quantile = param.Number(.99, bounds=(.5, 1))
    
    # Embeddings
    pca_data = param.Dict()
    tsne_data = param.Array()
    umap_data = param.Array()
    no_pca_comps = param.Integer(10, bounds=(1, None))
    
    EMPTY_DATA_2D = [[np.NaN, np.NaN], [np.NaN, np.NaN]]
    EMPTY_DATA_3D = [[[np.NaN, np.NaN], [np.NaN, np.NaN]], [[np.NaN, np.NaN], [np.NaN, np.NaN]]]
    exrds = xr.Dataset()
    eds = hv.Dataset([])
    
    calculate_button = pn.widgets.Button(name="Calculate embeddings")
    
    current_sel = param.Array()
    
    embedding_data = param.DataFrame()

    # Data processing
    @param.depends("filename", watch=True)
    def import_data(self):
        """Import data when the selected file changes"""
        spectral_info, spectral_data = parseFile(self.filename)
        
        self.band_min = int(min( abs(spectral_info["zStart"]), abs(spectral_info["zStart"] + spectral_info["zDelta"] * (spectral_info["zLength"] - 1)) ))
        self.band_max = int(max( abs(spectral_info["zStart"]), abs(spectral_info["zStart"] + spectral_info["zDelta"] * (spectral_info["zLength"] - 1)) ))
        self.band_change = int(spectral_info["zDelta"])

        self.wavenumbers = np.arange(self.band_max, self.band_min+self.band_change, self.band_change)
        self.param.band.objects = self.wavenumbers
        self.band = self.param.band.objects[-1]
        
        self.spectral_data = spectral_data
                
    @param.depends("spectral_data", "do_trans_to_abs", watch=True)
    def transform_data(self):
        """Transform transmission into absorbance"""
        if self.do_trans_to_abs:
            # Transform transmission into absorbance
            self.raw_data = -np.log10(self.spectral_data / 100)
            
            # Remove NaN values from logarithm of negative values
            np.nan_to_num(self.raw_data, copy=False, nan=np.nanmin(self.raw_data))
        
        else:
            self.raw_data = self.spectral_data
            
    @param.depends("raw_data", "do_baseline_shift", "shifting_band", "shifting_range", watch=True)
    def baseline_shift(self):
        """Carry out a baseline shift"""
        if self.do_baseline_shift:
            shift_band_1 = self._get_band(self.shifting_band - self.shifting_range)
            shift_band_2 = self._get_band(self.shifting_band + self.shifting_range)
            shift_band_low = min(shift_band_1, shift_band_2)
            shift_band_high = max(shift_band_1, shift_band_2)

            shift_abs = self.raw_data[..., np.arange(shift_band_low, shift_band_high)].mean(-1)
            self.all_data = self.raw_data - np.repeat(shift_abs[..., np.newaxis], self.raw_data.shape[-1], axis=-1)
            
        else:
            self.all_data = self.raw_data
        
    @param.depends("all_data", "do_normalisation", "normalisation_band", "normalisation_range", watch=True)
    def normalise(self):
#         print("Normalising data")
        """Add data to the correct data containers and normalise it"""
        
        # Do nothing if there is not data yet
        if self.all_data is not None:
            if self.do_normalisation:
                norm_band_1 = self._get_band(self.normalisation_band - self.normalisation_range)
                norm_band_2 = self._get_band(self.normalisation_band + self.normalisation_range)
                norm_band_low = min(norm_band_1, norm_band_2)
                norm_band_high = max(norm_band_1, norm_band_2)
                
                norm_mean = np.median(self.all_data[..., np.arange(norm_band_low, norm_band_high)].mean(-1))

                self.all_norm_data = np.divide(self.all_data, np.repeat(self.all_data[..., np.arange(norm_band_low, norm_band_high)].mean(-1)[..., np.newaxis], self.all_data.shape[-1], axis=-1)) * norm_mean
                self.data.exrds = xr.Dataset(
                    data_vars = dict(absorbance=(["x", "y", "band"], self.all_norm_data)),
                    coords = dict(x=np.arange(self.all_norm_data.shape[0]), y=np.arange(self.all_norm_data.shape[1]), band=self.wavenumbers),
                ).stack(idx=("x", "y"))
            else:
                self.exrds = xr.Dataset(
                    data_vars = dict(absorbance=(["x", "y", "band"], self.all_data)),
                    coords = dict(x=np.arange(self.all_data.shape[0]), y=np.arange(self.all_data.shape[1]), band=self.wavenumbers),
                ).stack(idx=("x", "y"))
#                 self.exrds = self.exrds.merge(exrds, overwrite_vars="absorbance", join="right")
            self.eds = hv.Dataset(self.exrds, ["band", "idx"])

    @param.depends("band_selector.value", watch=True)
    def update_band(self):
        # Initialise band_selector first with data
        if self.band_selector.value != 0:
            self.band = self.band_selector.value
    
    @param.depends("shifting_band_selector.value", watch=True)
    def update_shifting_band(self):
        self.shifting_band = self.shifting_band_selector.value
    
    @param.depends("normalisation_band_selector.value", watch=True)
    def update_normalisation_band(self):
        self.normalisation_band = self.normalisation_band_selector.value
    
    @param.depends("band", watch=True)
    def calculate_band_index(self):
        self.band_index = self._get_band(self.band)
    
    @param.depends("spectral_data", "shifting_range", "normalisation_range", watch=True)
    def update_band_selectors(self):
        self.band_selector.start = self.band_min
        self.band_selector.stop = self.band_max
        self.band_selector.step = abs(self.band_change)
        
        self.shifting_band_selector.start = self.band_min + self.shifting_range + abs(self.band_change)
        self.shifting_band_selector.stop = self.band_max - self.shifting_range
        self.shifting_band_selector.step = abs(self.band_change)
        
        self.normalisation_band_selector.start = self.band_min + self.normalisation_range + abs(self.band_change)
        self.normalisation_band_selector.stop = self.band_max - self.normalisation_range
        self.normalisation_band_selector.step = abs(self.band_change) 
    
    def view(self):
        file_select = pn.widgets.TextInput.from_param(self.param.filename, value=r"C:\Users\gieri\OneDrive\Documenten\Course manuals\Stage\Nachtwacht\SK-C-5_047\SK-C-5_047_FTIR-ATR_4cm_2021-04-12_raw.fs")
        ta_toggle = pn.widgets.Checkbox.from_param(self.param.do_trans_to_abs)
        baseline_toggle = pn.widgets.Checkbox.from_param(self.param.do_baseline_shift)
        normalisation_toggle = pn.widgets.Checkbox.from_param(self.param.do_normalisation)
        shift_range = pn.widgets.IntInput.from_param(self.param.shifting_range)
        norm_range = pn.widgets.IntInput.from_param(self.param.normalisation_range)
        quantile_selector = pn.widgets.FloatInput.from_param(self.param.quantile)
        
        return pn.Column(
            file_select,
            pn.Row(ta_toggle, baseline_toggle, normalisation_toggle),
            pn.Row(self.shifting_band_selector, self.normalisation_band_selector),
            pn.Row(shift_range, norm_range),
            pn.Row(self.band_selector, quantile_selector) 
        )
    
    def calculate_quantile(self, data, quantile=.99):
        lower_q = np.nanquantile(data, 1 - quantile)
        upper_q = np.nanquantile(data, quantile)

        return np.clip(data, lower_q, upper_q)
    
    def get_coordinates(self, data):
        """
        Create a coordinate space from the provided data.
        
        Args:
            data (np.array): spectral data.
            
        Returns:
            xy (np.array): 2D array with the x- and y- coordinates in 
                a x times y grid.
        """
        x = [x % data.shape[1] for x in range(np.product(data.shape[:2]))]
        y = [int(np.floor(y / data.shape[1])) for y in range(np.product(data.shape[:2]))]
        
        return x, y
            
    def _get_band(self, band):
        return int( (band - self.band_max) / self.band_change )
    

class RegionSelection(param.Parameterized):
    sample_selection = hv.link_selections.instance(unselected_alpha=.5, cross_filter_mode="overwrite")
    
    selection_confirm = pn.widgets.Toggle(name="Confirm selection")
    
    EMPTY_MAP = ([0, 10_000], [0, 10_000], [[None, None], [None, None]])
    data_pipe = hv.streams.Pipe(EMPTY_MAP)
    sel_data_pipe = hv.streams.Pipe(EMPTY_MAP)
    
    def __init__(self, datacontainer):
        self.data = datacontainer
        super().__init__()    
    
    
    def create_selection_plot(self):
         return self.sample_selection(
             hv.DynamicMap(hv.Image, streams=[self.data_pipe]).opts(
                 hv.opts.Image(
                     cmap="viridis", framewise=True, active_tools=["box_select"], title="Region selection",
                     aspect="equal", frame_height=int(2*size),
                 ),
             )
         ).redim(x="selection_x", y="selection_y")
    
    def show_selection(self):
        return hv.DynamicMap(
            hv.Image, streams=[self.sel_data_pipe]
        ).opts(
            cmap="viridis", framewise=True, tools=["hover"],
            title="Selected region",  aspect="equal", frame_height=int(2*size), axiswise=True,
        ).redim(x="sel_x", y="sel_y").redim.label(sel_x="x", sel_y="y", z="Absorbance").apply.opts(
            xlim = self.data.sel_plot_opts.param.xlim, ylim = self.data.sel_plot_opts.param.ylim, 
        )

    @param.depends("data.all_data", "data.all_norm_data", "data.do_normalisation", "data.band", "data.quantile", watch=True)
    def update_data_pipe(self):
        # Skip if there is no data yet
        if self.data.all_data is not None:
            if self.data.do_normalisation:
                data = self.data.calculate_quantile(self.data.all_norm_data[..., self.data.band_index], self.data.quantile)
                
                self.data_pipe.send(
                    (np.arange(data.shape[0]), 
                     np.arange(data.shape[1]), 
                     data) 
                )
                
            else:
                data = self.data.calculate_quantile(self.data.all_data[..., self.data.band_index], self.data.quantile)
                
                self.data_pipe.send(
                    (np.arange(data.shape[1]), 
                     np.arange(data.shape[0]), 
                     data) 
                )
            
    @param.depends("data.sel_data", "data.band", "data.quantile", watch=True)
    def update_sel_data_pipe(self):
        if self.data.sel_data is None:
            self.sel_data_pipe.send(self.EMPTY_MAP)
        
        else:
            data = self.data.calculate_quantile(self.data.sel_data[..., self.data.band_index], self.data.quantile)
                
            self.sel_data_pipe.send(
                (np.arange(data.shape[1]),
                 np.arange(data.shape[0]),
                 data) 
            )
            
            self.data.sel_plot_opts.xlim = (-0.5, data.shape[1]-0.5)
            self.data.sel_plot_opts.ylim = (-0.5, data.shape[0]-0.5)
            
    @param.depends("sample_selection.selection_expr", watch=True)
    def update_bounds(self):
        sel_expr = str(self.sample_selection.selection_expr)
        self.data.x_min = max(0, round(float(re.search(r"'x'\)>=(\-*\d+\.*\d*)", sel_expr).group(1))))
        self.data.x_max = min(self.data.all_data.shape[1], round(float(re.search(r"'x'\)<=(\-*\d+\.*\d*)", sel_expr).group(1))))
        self.data.y_min = max(0, round(float(re.search(r"'y'\)>=(\-*\d+\.*\d*)", sel_expr).group(1))))
        self.data.y_max = min(self.data.all_data.shape[0], round(float(re.search(r"'y'\)<=(\-*\d+\.*\d*)", sel_expr).group(1))))
    
    @param.depends("selection_confirm.value", watch=True)
    def toggle_selection_button(self):
        if self.selection_confirm.value:
            self.selection_confirm.name = "Change selection"
            
        else:
            self.selection_confirm.name = "Confirm selection"
    
    @param.depends("selection_confirm.value", "data.all_data", "data.all_norm_data", watch=True)
    def update_selection_data(self):
        if self.selection_confirm.value:
            if self.data.do_normalisation:
                sel_data = self.data.all_norm_data[self.data.y_min:self.data.y_max, self.data.x_min:self.data.x_max, :]
            else:
                sel_data = self.data.all_data[self.data.y_min:self.data.y_max, self.data.x_min:self.data.x_max, :]
            
            # Replace the old data with the new selection
            self.data.exrds = xr.Dataset(
                data_vars = dict(absorbance=(["x", "y", "band"], sel_data)),
                coords = dict(x=np.arange(sel_data.shape[0]), y=np.arange(sel_data.shape[1]), band=self.data.wavenumbers),
            ).stack(idx=("x", "y"))
            self.data.eds = hv.Dataset(self.data.exrds, ["band", "idx"])
            self.data.sel_data = sel_data
        else:
            self.data.sel_data = None
        
    def view(self):
        selection_plot = self.create_selection_plot()
        selected_plot = self.show_selection()
        confirm_selection_button = self.selection_confirm
        
        return pn.Column(
            pn.Row(selection_plot, selected_plot),
            confirm_selection_button
        )
    

class EmbeddingsProducer(param.Parameterized):

    do_pca = param.Boolean()
    do_tsne = param.Boolean()
    do_umap = param.Boolean()

    pca_params = param.Dict(dict(
        whiten=False,
        svd_solver='auto',
        tol=0.0,
        iterated_power='auto',
        random_state=None,
    ))
    
    tsne_distance_metric = param.Selector({
        "Euclidean": "euclidean",
        "Pre-computed": "precomputed",
        "Cosine": "cosine",
        "Correlation": "correlation",
        "Manhattan/City block": "cityblock",
        "Haversine": "haversine",
        "Chebyshev": "chebyshev",
        "Minkowski": "minkowsky",
        "Canberra": "canberra",
        "Bray-Curtis": "braycurtis",
        "Mahalanobis": "mahalanobis",
        "Weighted Minkowsky": "wminkowsky",
        "Std. Euclidean": "seuclidean",
    })
    tsne_params = param.Dict(dict(
        n_components=2,
        perplexity=30.0,
        early_exaggeration=12.0,
        learning_rate=200.0,
        n_iter=1000,
        n_iter_without_progress=300,
        min_grad_norm=1e-07,
        init='random',
        verbose=0,
        random_state=None,
        method='barnes_hut',
        angle=0.5,
        n_jobs=None,
        square_distances='legacy',
    ))
    
    umap_distance_metric = param.Selector({
        "Euclidean": "euclidean",
        "Cosine": "cosine",
        "Correlation": "correlation",
        "Manhattan/City block": "cityblock",
        "Haversine": "haversine",
        "Chebyshev": "chebyshev",
        "Minkowski": "minkowsky",
        "Canberra": "canberra",
        "Bray-Curtis": "braycurtis",
        "Mahalanobis": "mahalanobis",
        "Weighted Minkowsky": "wminkowsky",
        "Std. Euclidean": "seuclidean",
    })
    umap_params = param.Dict(dict(
        n_neighbors=15, n_components=2, metric_kwds=None, output_metric='euclidean', output_metric_kwds=None,
        n_epochs=None, learning_rate=1.0, init='spectral', min_dist=0.1, spread=1.0, low_memory=True,
        n_jobs=-1, set_op_mix_ratio=1.0, local_connectivity=1.0, repulsion_strength=1.0, negative_sample_rate=5,
        transform_queue_size=4.0, a=None, b=None, random_state=None, angular_rp_forest=False, target_n_neighbors=-1,
        target_metric='categorical', target_metric_kwds=None, target_weight=0.5, transform_seed=42,
        transform_mode='embedding', force_approximation_algorithm=False, verbose=False, unique=False,
        densmap=False, dens_lambda=2.0, dens_frac=0.3, dens_var_shift=0.1, output_dens=False, 
        disconnection_distance=None,
    ))
    
    
    def __init__(self, datacontainer):
        self.data = datacontainer
        super().__init__()
    
    @param.depends("data.calculate_button.clicks", watch=True)
    def calculate_embeddings(self):
        print("Calculating embeddings")
        
        # Skip calculation if there is no data yet
        if self.data.all_data is not None:
            # Flatten the data into a 2D matrix
            # If no data is selected, take all data
            if self.data.sel_data is None:
                flattened_data = self.data.all_data.reshape(-1, self.data.all_data.shape[-1])
            else:
                flattened_data = self.data.sel_data.reshape(-1, self.data.sel_data.shape[-1])

            # PCA
            if self.do_pca:
                pca = PCA(n_components=self.data.no_pca_comps, **self.pca_params)

                pca_data = dict(
                    scores = pca.fit_transform(flattened_data),
                    loadings = pca.components_.T * np.sqrt(pca.explained_variance_),
                    expl_var = pca.explained_variance_ratio_,
                )

#                 columns = [f"PC {i+1}" for i in range(pca_data["scores"].shape[1])]
                self.data.exrds = self.data.exrds.assign({
                    f"PC {i+1}": ("idx", pca_data["scores"][:, i])
                    for i in range(pca_data["scores"].shape[1])
                })
                self.data.eds = hv.Dataset(self.data.exrds, ["band", "idx"])

                self.data.pca_data = pca_data
            else:
                self.data.pca_data = None

            # t-SNE
            if self.do_tsne:
                reducer = TSNE(metric=self.tsne_distance_metric, **self.tsne_params)
                tsne_data = reducer.fit_transform(flattened_data)
                self.data.exrds = self.data.exrds.assign({
                    f"t-SNE component {i+1}": ("idx", tsne_data[:, i])
                    for i in range(tsne_data.shape[1])
                })
                self.data.eds = hv.Dataset(self.data.exrds, ["band", "idx"])
                
                self.data.tsne_data = tsne_data
            else:
                self.data.tsne_data = None

            # UMAP
            if self.do_umap:
                reducer = umap.UMAP(metric=self.umap_distance_metric, **self.umap_params)
                umap_data = reducer.fit_transform(flattened_data)
                self.data.exrds = self.data.exrds.assign({
                    f"UMAP component {i+1}": ("idx", umap_data[:, i])
                    for i in range(umap_data.shape[1])
                })
                self.data.eds = hv.Dataset(self.data.exrds, ["band", "idx"])
                
                self.data.umap_data = umap_data
            else:
                self.data.umap_data = None
    
    def view(self):
        do_pca = pn.widgets.Checkbox.from_param(self.param.do_pca, name="PCA")
        pca_comp = pn.widgets.IntInput.from_param(self.data.param.no_pca_comps, name="No. of PCA components")
        pca_params = pn.widgets.LiteralInput.from_param(self.param.pca_params, name="Other PCA parameters (dict)")
        do_tsne = pn.widgets.Checkbox.from_param(self.param.do_tsne, name="t-SNE")
        tsne_dist = pn.widgets.Select.from_param(self.param.tsne_distance_metric, name="t-SNE distance metric")
        tsne_params = pn.widgets.LiteralInput.from_param(self.param.tsne_params, name="Other t-SNE parameters (dict)")
        do_umap = pn.widgets.Checkbox.from_param(self.param.do_umap, name="UMAP")
        umap_dist = pn.widgets.Select.from_param(self.param.umap_distance_metric, name="UMAP distance metric")
        umap_params = pn.widgets.LiteralInput.from_param(self.param.umap_params, name="Other UMAP parameters (dict)")
        cb = self.data.calculate_button
        
        return pn.Column(
                
            pn.Row(do_pca, pca_comp, pca_params),
            pn.Row(do_tsne, tsne_dist, tsne_params),
            pn.Row(do_umap, umap_dist, umap_params,),
            cb,
        )


class ExplorationDashboard(param.Parameterized):
    
    pcx = param.Integer(1)
    pcy = param.Integer(2)
#     sel_idxs = param.Array()
    sel_bool = param.Array()
    
    pca_sel_pipe = hv.streams.Pipe([])
    pca_nonsel_pipe = hv.streams.Pipe([])
    tsne_sel_pipe = hv.streams.Pipe([])
    tsne_nonsel_pipe = hv.streams.Pipe([])
    umap_sel_pipe = hv.streams.Pipe([])
    umap_nonsel_pipe = hv.streams.Pipe([])
    sample_sel_pipe = hv.streams.Pipe([])
    sample_nonsel_pipe = hv.streams.Pipe([])
    
    pca_opts = Opts(xlabel="PC 1", ylabel="PC 2")
    tsne_opts = Opts()
    umap_opts = Opts()
    sample_opts = Opts()
    
    ls1 = hv.link_selections.instance()
    ls2 = hv.link_selections.instance()
    ls3 = hv.link_selections.instance()
    ls4 = hv.link_selections.instance()
    lss = [ls1, ls2, ls3, ls4]
    
    def __init__(self, datacontainer):
        self.data = datacontainer
        self.param.pcx.bounds = (1, self.data.no_pca_comps)
        self.param.pcy.bounds = (1, self.data.no_pca_comps)
        
        super().__init__()
    
    clear_selection_button = pn.widgets.Button(name="Clear selection")
    # clear_selection_button.param.watch(clear_selection, 'clicks')

    @param.depends("clear_selection_button.clicks", watch=True)
    def clear_selection(self):
        self.data.current_sel = None
    
    def get_pca_plot(self, data):
        if self.pca_opts.xlabel in data:
            return hv.Scatter(data.set_coords(self.pca_opts.xlabel), kdims=[self.pca_opts.xlabel], vdims=[self.pca_opts.ylabel], )
        else:
            return hv.Scatter([], kdims=[self.pca_opts.xlabel], vdims=[self.pca_opts.ylabel], )
    
    def plot_pca(self):
        return self.ls1(
            dynspread(rasterize(hv.DynamicMap(self.get_pca_plot, streams=[self.pca_sel_pipe]))).opts(cmap="kbc_r") *
            dynspread(rasterize(hv.DynamicMap(self.get_pca_plot, streams=[self.pca_nonsel_pipe]))).opts(cmap="reds", alpha=0.5)
        ).opts(
                hv.opts.Scatter(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="PCA" ),
                hv.opts.Points(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="PCA" ),
                hv.opts.Image(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="PCA" ),
                hv.opts.RGB(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="PCA" ),
        ).apply.opts(xlabel=self.pca_opts.param.xlabel, ylabel=self.pca_opts.param.ylabel, 
                     xlim=self.pca_opts.param.xlim, ylim=self.pca_opts.param.ylim
        )
     
    def plot_tsne(self):
        return self.ls2(
            dynspread(rasterize(hv.DynamicMap(partial(hv.Scatter, kdims="t-SNE component 1", vdims="t-SNE component 2"), streams=[self.tsne_sel_pipe]))).opts(cmap="kbc_r") *
            dynspread(rasterize(hv.DynamicMap(partial(hv.Scatter, kdims="t-SNE component 1", vdims="t-SNE component 2"), streams=[self.tsne_nonsel_pipe]))).opts(cmap="reds", alpha=0.5)
        ).opts(
            hv.opts.Image(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="t-SNE embedding", ),
            hv.opts.RGB(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="t-SNE embedding", ),
        ).apply.opts(xlim=self.tsne_opts.param.xlim, ylim=self.tsne_opts.param.ylim)
    
    def plot_umap(self):
        return self.ls3(
            dynspread(rasterize(hv.DynamicMap(partial(hv.Scatter, kdims=["UMAP component 1"], vdims=["UMAP component 2"]), streams=[self.umap_sel_pipe]))).opts(cmap="kbc_r") *
            dynspread(rasterize(hv.DynamicMap(partial(hv.Scatter, kdims=["UMAP component 1"], vdims=["UMAP component 2"]), streams=[self.umap_nonsel_pipe]))).opts(cmap="reds", alpha=0.5)
        ).opts(
            hv.opts.Image(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="UMAP embedding", ),
            hv.opts.RGB(framewise=True, frame_width=int(1*size), frame_height=int(1*size), title="UMAP embedding", ),
        ).apply.opts(xlim=self.umap_opts.param.xlim, ylim=self.umap_opts.param.ylim)
    
    def plot_sample(self):
        return self.ls4(
            hv.DynamicMap(partial(hv.Image, kdims=["x", "y"], vdims=["absorbance"]), streams=[self.sample_sel_pipe]).opts(cmap="kbc_r") *
            hv.DynamicMap(partial(hv.Image, kdims=["x", "y"], vdims=["absorbance"]), streams=[self.sample_nonsel_pipe]).opts(cmap="reds", alpha=0.2)
        ).opts(
            hv.opts.Image(framewise=True, frame_width=int(1*size), frame_height=int(1*size), clipping_colors={"NaN": "transparent"}, invert_axes=True),
        ).apply.opts(
            title = self.sample_opts.param.title, aspect=self.sample_opts.param.aspect,
            xlim = self.sample_opts.param.xlim, ylim = self.sample_opts.param.ylim, 
        )
    
#     @param.depends("ls1.selection_expr", watch=True)
    def update_selection1(self):
        if self.ls1.selection_expr is not None:
            self.update_selection_indices(self.ls1.selection_expr)
            
            self.ls1.selection_expr = None
    
    @param.depends("ls2.selection_expr", watch=True)
    def update_selection2(self):
        if self.ls2.selection_expr is not None:
            self.update_selection_indices(self.ls2.selection_expr)
            
            self.ls2.selection_expr = None
            
    @param.depends("ls3.selection_expr", watch=True)
    def update_selection3(self):
        if self.ls3.selection_expr is not None:
            self.update_selection_indices(self.ls3.selection_expr)
            
            self.ls3.selection_expr = None
            
    @param.depends("ls4.selection_expr", watch=True)
    def update_selection4(self):
        if self.ls4.selection_expr is not None:
            self.update_selection_indices(self.ls4.selection_expr)
            
            self.ls4.selection_expr = None
    
    # Implicitly depends on self.data.current_sel
    def update_selection_indices(self, selection_expr=None):
        # Create boolean array with the current selection indices as True 
        new_sel_idxs_multi = self.data.eds.select(selection_expr).data.idx.data.astype("i,i")
        all_idxs = dc.exrds.idx.data.astype("i,i")
        new_sel_bool = np.isin(all_idxs, new_sel_idxs_multi)
        
        # If no selection has been made yet, set the selection to the current selection
        # else get the intersection between the previous and current selection
        # N.B. current_sel is in the DataContainer to prevent looping from implicit dependency
        if self.data.current_sel is None:
            updated_sel = new_sel_bool
        else:
            updated_sel = np.logical_and(self.data.current_sel, new_sel_bool)
        
        # Check if the selected indexes have changed, if so update the arrays
        # that contain the selections and update the plots
        if not np.equal(updated_sel, self.data.current_sel).all():
            self.data.current_sel = updated_sel
    
    @param.depends("data.pca_data", "data.tsne_data", "data.umap_data", watch=True)
    # Implicitly depends on self.data.current_sel
    def update_embeddings_pipes(self):
        print("Updating embeddings")

        data = self.data.exrds.drop(["absorbance", "band"])
        
        sel = data.unstack()
        if self.data.current_sel is not None:
            # Split selected and non-selected data points and send them to the appropriate pipes
            sel = data[dict(idx=self.data.current_sel)].unstack()
            nonsel = data[dict(idx=~self.data.current_sel)].unstack()
        
        if self.data.pca_data is None:
            pass
#             if self.ls.selection_expr is None:
#                 self.pca_data_pipe.send([])
        else:
            pc_names = [f"PC {i+1}" for i in range(self.data.pca_data["expl_var"].shape[0])]

            self.pca_sel_pipe.send(sel[pc_names])
            if self.data.current_sel is None:
                self.pca_nonsel_pipe.send([])
            else:
                self.pca_nonsel_pipe.send(nonsel[pc_names])
                
            self.pca_opts.xlim = (self.data.exrds[self.pca_opts.xlabel].min().item(), self.data.exrds[self.pca_opts.xlabel].max().item())
            self.pca_opts.ylim = (self.data.exrds[self.pca_opts.ylabel].min().item(), self.data.exrds[self.pca_opts.ylabel].max().item())
            
        if self.data.tsne_data is None:
            pass
#             if self.ls.selection_expr is None:
#                 self.tsne_data_pipe.send([])
        else:
            tsne_names = [f"t-SNE component {i+1}" for i in range(self.data.tsne_data.shape[1])]
        
            self.tsne_sel_pipe.send(sel[tsne_names].set_coords("t-SNE component 1"))
            if self.data.current_sel is None:
                self.tsne_nonsel_pipe.send([])
            else:
                self.tsne_nonsel_pipe.send(nonsel[tsne_names].set_coords("t-SNE component 1"))
        
            self.tsne_opts.xlim = (self.data.tsne_data[:, 0].min(), self.data.tsne_data[:, 0].max())
            self.tsne_opts.ylim = (self.data.tsne_data[:, 1].min(), self.data.tsne_data[:, 1].max())
        
        if self.data.umap_data is None:
            pass
#             if self.ls.selection_expr is None:
#                 self.umap_sel_pipe.send([])
        else:
            umap_names = [f"UMAP component {i+1}" for i in range(self.data.umap_data.shape[1])]
            
            self.umap_sel_pipe.send(sel[umap_names].set_coords("UMAP component 1"))
            if self.data.current_sel is None:
                self.umap_nonsel_pipe.send([])
            else:
                self.umap_nonsel_pipe.send(nonsel[umap_names].set_coords("UMAP component 1"))

            self.umap_opts.xlim = (self.data.umap_data[:, 0].min(), self.data.umap_data[:, 0].max())
            self.umap_opts.ylim = (self.data.umap_data[:, 1].min(), self.data.umap_data[:, 1].max())     
            
    @param.depends("data.sel_data", "data.band", "data.current_sel", watch=True)
    def update_sample_pipe(self):
        if "absorbance" in self.data.exrds:
            data = self.data.exrds.sel(band=self.data.band)
            
            sel = data.absorbance.unstack()
            nonsel = []
            if self.data.current_sel is not None:
                # Split selected and non-selected data points and send them to the appropriate pipes
                sel = data.where(self.data.current_sel).absorbance.unstack()
                nonsel = data.where(~self.data.current_sel).absorbance.unstack()
            
            self.sample_opts.title = f"Absorbance at {self.data.band} cm⁻¹"
            self.sample_opts.aspect = "equal"
            self.sample_opts.xlim = (data.x.min().item(), data.x.max().item())
            self.sample_opts.ylim = (data.y.min().item(), data.y.max().item())

            self.sample_sel_pipe.send(sel)
            self.sample_nonsel_pipe.send(nonsel)

        else:
            self.sample_sel_pipe.send([])
            self.sample_nonsel_pipe.send([])
    
    def view(self):
        pcx = pn.widgets.IntInput.from_param(self.param.pcx, name="Plot PC on x-axis")
        pcy = pn.widgets.IntInput.from_param(self.param.pcy, name="Plot PC on y-axis")
        
        return (
                pn.Column(
                pn.Row(pcx, pcy),
                self.clear_selection_button,
                hv.Layout([
                    self.plot_tsne(),
                    self.plot_umap(),
                    self.plot_pca(),
                ]).cols(3).opts(shared_axes=False),
                self.plot_sample()
            )
        )
    
        
dc = DataContainer()
dc_view = dc.view()

rs = RegionSelection(dc)
rs_view = rs.view()

ep = EmbeddingsProducer(dc)
ep_view = ep.view()

ed = ExplorationDashboard(dc)
ed_view = ed.view()

pn.Column(
    dc_view,
    rs_view,
    ep_view,
    ed_view,
)

#### Dashboard bottom

In [ ]:
dc.eds.select(ed.ls1.selection_expr)

In [ ]:
ed.ls1.selection_expr

In [ ]:
ed.update_sample_pipe()

In [ ]:
dc.sel_plot_opts.param.xlim.

In [ ]:
dc.current_sel

In [ ]:
dc.exrds.sel(band=1536).where(~dc.current_sel)

In [ ]:
data = dc.exrds.sel(band=1536).where(~dc.current_sel).unstack()
hv.Image(data, ["x", "y"], "absorbance")

In [ ]:
hv.Image(dc.current_sel.reshape(dc.sel_data.shape[:2]))

In [ ]:
ed.ls1

In [ ]:
ed.ls1.selection_expr

In [ ]:
x = dc.exrds.stack(idx=["x", "y"])
y = hv.Dataset(x, ["idx", "band"])

In [ ]:
x

In [ ]:
y.select(ed.ls1.selection_expr)

In [ ]:
z = y.select(ed.ls1.selection_expr)

In [ ]:
x.idx.data

In [ ]:
np.isin(x.idx.data, z.data.idx.data)

In [ ]:
np.array([(92, 60), (95, 17)])

In [ ]:
si = z.data.idx.data.astype("i,i")
ai = dc.exrds.stack(idx=["x", "y"]).idx.data.astype("i,i")

In [ ]:
np.isin(ai, si)

In [ ]:
np.where(np.isin(ai, si))

In [ ]:
s.unstack()

In [ ]:
x.where(z.data)

In [ ]:
ai

In [ ]:
si

In [ ]:
np.isin(ai, si).all(1).shape

In [ ]:
dc.eds[.]

In [ ]:
dc.eds.select(ed.ls1.selection_expr)

In [ ]:
data

In [ ]:
data = ed.data.exrds.drop(["absorbance", "band"]).stack(idx=("x", "y"))
pc_names = [f"PC {i+1}" for i in range(ed.data.pca_data["expl_var"].shape[0])]

# self.pca_sel_pipe.send(sel[pc_names])
data[dict(idx=np.repeat(True, len(data.idx)))].unstack()[pc_names]

In [ ]:
dc.exrds[a]

In [ ]:
ds = dc.exrds[["UMAP component 1", "UMAP component 2"]]
ds

In [ ]:
rasterize(hv.Scatter(ds.set_coords("UMAP component 1"), "UMAP component 1", "UMAP component 2"))

In [ ]:
data = ed.data.exrds.absorbance.sel(band=ed.data.band).stack(idx=("x", "y"))

In [ ]:
ed.data.embedding_data[["x", "y", "abs_1536"]]

In [ ]:
data.unstack()[t]

In [ ]:
dc.embedding_data

In [ ]:
t = np.isin(np.arange(ed.data.embedding_data.index.stop), ed.test)

In [ ]:
ed.data.embedding_data[np.isin(np.arange(ed.data.embedding_data.index.stop), ed.test)]

In [ ]:
ed.data.embedding_data[ed.data.embedding_data["idx"].isin(ed.test)]

In [ ]:
hv.Image(data.unstack("idx"), ["x", "y"], "absorbance")[np.isin(np.arange(ed.data.embedding_data.index.stop), ed.test)]

In [ ]:
data[np.isin(np.arange(ed.data.embedding_data.index.stop), ed.test)]

In [ ]:
data.where(data["idx"].isin(ed.test2)).unstack("idx")

In [ ]:
ed.test

In [ ]:
ed.sel_idxs = ed.test

In [ ]:
ed.sel_idxs

In [ ]:
ed.test

In [ ]:
dc.eds.select(ed.ls1.selection_expr).data.index

In [ ]:
ed.lss

In [ ]:
ed.sel_idxs

In [ ]:
f = rasterize(hv.Scatter(dc.embedding_data, "UMAP component 1", "UMAP component 2")).opts(tools=["box_select"])
sel = hv.streams.Selection1D(source=f)
f

In [ ]:
sel

In [ ]:
dc.sel_plot_opts

In [ ]:
dc.exrds

In [ ]:
data.stack(idx=("x", "y"))

In [ ]:
ed.sel_idxs

In [ ]:
ds = ed.data.exrds.absorbance.stack(idx=("x", "y"))
data;
sel = ds.where(ds["idx"].isin(ed.sel_idxs)).unstack("idx")
sel

In [ ]:
hv.Image(dc.exrds.sel(band=690), ["x", "y"], "absorbance")

In [ ]:
xys = np.array([xy[idx] for idx in ed.sel_idxs])
xys

In [ ]:
x = [x % dc.all_data.shape[1] for x in range(np.product(dc.all_data.shape[:2]))]
y = [int(np.floor(y / dc.all_data.shape[1])) for y in range(np.product(dc.all_data.shape[:2]))]
xy = np.stack([x, y]).T

In [ ]:
dc.exrds["idx"].isin(ed.sel_idxs)

In [ ]:
sl = hv.link_selections.instance()

In [ ]:
sl.selection_expr = None

In [ ]:
sl((hv.Image(dc.exrds.sel(band=1536), ["x", "y"], "absorbance")))

In [ ]:
sl

In [ ]:
sl(rasterize(hv.Scatter((dc.exrds["PC 1"], dc.exrds["PC 2"]), "PC 1", "PC 2")))

In [ ]:
sl;

In [ ]:
dc.eds.data.x

In [ ]:
dc.eds.select(sl.selection_expr).data.idx

In [ ]:
dc.exrds

In [ ]:
hv.Image(dc.exrds.stack(idx=("x", "y")).where(dc.exrds["idx"].isin(ed.sel_idxs)).unstack("idx").sel(band=1536), ["x", "y"], "absorbance")

In [ ]:
sel = self.data.embedding_data[self.data.embedding_data["idx"].isin(self.sel_idxs)]

In [ ]:
dat = deepcopy(dc.exrds)

In [ ]:
dc.umap_data

In [ ]:
dc.umap_data[:,0]

In [ ]:
dat = dat.assign(UMAP=(["idx", "comp1", "comp2"], np.ones(dc.umap_data.shape)))

In [ ]:
dat = dat.stack(idx=("x", "y"))

In [ ]:
dat

In [ ]:
hv.Image(dat, ["idx", "comp"], "UMAP")

In [ ]:
sl(rasterize(hv.Scatter(dat.UMAP.data)))

In [ ]:
sl

In [ ]:
rasterize(hv.Scatter(dat.UMAP))

In [ ]:
dat

In [ ]:
a = dc.exrds.sel(band=1536)
a

In [ ]:
a.absorbance.shape

In [ ]:
ed.sel_idxs

In [ ]:
np.isin(np.arange(ed.data.embedding_data.index.stop), ed.sel_idxs)

In [ ]:
selected_indexes_bool = np.isin(np.arange(ed.data.embedding_data.index.stop), ed.sel_idxs)
sib_reshaped = selected_indexes_bool.reshape(a.x.max().data+1, a.y.max().data+1)

In [ ]:
sib_reshaped.shape

In [ ]:
xr.DataArray(data=sib_reshaped, dims=["x", "y"])

In [ ]:
a.absorbance.where(xr.DataArray(data=sib_reshaped, dims=["x", "y"]))

In [ ]:
hv.Image(a.absorbance.where(xr.DataArray(data=sib_reshaped, dims=["x", "y"])), ["x", "y"], "absorbance")

In [ ]:
hv.Image(dc.exrds.sel(band=1536), ["x", "y"]).opts(
    cmap="viridis", clipping_colors={"NaN": "transparent"}
).select(idx=ed.sel_idxs)

In [ ]:
sel_idxs = ed.data.eds.select(ed.ls1.selection_expr).data.idx.to_numpy()
nonsel_idxs = np.array(list(set(np.arange(ed.data.embedding_data.index.stop)) - set(sel_idxs)))
sel_idxs, nonsel_idxs

In [ ]:
idxspd = hv.Dataset(ed.data.embedding_data).select(ed.ls1.selection_expr).data.idx.to_numpy()
idxspd

In [ ]:
import datashader as ds

In [ ]:
ed.data.embedding_data[ed.data.embedding_data["idx"].isin(sel_idxs)]

In [ ]:
ed.data.exrds

In [ ]:
hv.Image(ed.data.exrds.sel(band=1536, idx=sel_idxs), ["x", "y"], "absorbance")

In [ ]:
sel_idxs = ed.data.eds.select(ed.ls1.selection_expr).data.idx.to_numpy()
nonsel_idxs = np.array(list(set(np.arange(ed.data.embedding_data.index.stop)) - set(sel_idxs)))

# sel = hv.Scatter(ed.data.embedding_data, "PC 1", "PC 2").iloc[sel_idxs]
# non_sel = hv.Scatter(ed.data.embedding_data, "PC 1", "PC 2").iloc[nonsel_idxs]
sel = hv.Scatter(ed.data.embedding_data, "UMAP component 1", "UMAP component 2").iloc[sel_idxs]
non_sel = hv.Scatter(ed.data.embedding_data, "UMAP component 1", "UMAP component 2").iloc[nonsel_idxs]

sel = datashade(hv.Points(ed.data.embedding_data, ["x", "y"], "abs_1536").iloc[sel_idxs])
non_sel = datashade(hv.Points(ed.data.embedding_data, ["x", "y"], "abs_1536").iloc[nonsel_idxs])

pn.Column(datashade(hv.NdOverlay({"sel": sel, "nonsel": non_sel}, "seli"), aggregator=ds.by("seli", ds.count())).opts(width=(2*size), height=(2*size)))

In [ ]:
hv.NdOverlay({"sel": sel, "all": alls}, "sel")

In [ ]:
pn.Row(sel, alls)

In [ ]:
dc.embedding_data["idx"]

In [ ]:
ed.update_embeddings_pipes()
ed.update_sample_pipe()

In [ ]:
df = pd.DataFrame({
#     'A': [0, 1, 2, 3, 4],
#     'B': [5, 6, 7, 8, 9],
    'C': ['a', 'b', 'c', 'd', 'e']
})
df

In [ ]:
df2 = pd.DataFrame({"A": [4, 3, 2, 1, 0], "B": [10, 11, 12, 13, 14]})
df[df2.columns] = df2
df

In [ ]:
dc.exrds

In [ ]:
dc.all_data.shape

In [ ]:
dc.embedding_data["idx"]

In [ ]:
s

In [ ]:
s = np.rot90(np.arange(np.product(dc.all_data.shape[:2])).reshape((dc.all_data.shape[:2])))
hv.Image(s)

In [ ]:
dc.exrds.absorbance

In [ ]:
dc.exrds = dc.exrds.assign_coords({"x": np.arange(dc.all_data.shape[1]), "y": np.arange(dc.all_data.shape[0]), "idx": (("x", "y"), np.rot90(np.arange(np.product(dc.sel_data.shape[:2])).reshape(dc.sel_data.shape[:2])))})

In [ ]:
dc.exrds = dc.exrds.assign_coords(idx=(("x", "y"), np.arange(np.product(dc.all_data.shape[:2])).reshape(dc.all_data.shape[:2])))

In [ ]:
ed.ls

In [ ]:
dc.embedding_data

In [ ]:
dc.embedding_data.dtypes.loc[["x", "y"]]

In [ ]:
dc.embedding_data["idx"] = dc.embedding_data.index
dc.embedding_data

In [ ]:
hv.Image(dc.embedding_data, ["x", "y"], "abs_1720")

In [ ]:
ed.ls

In [ ]:
ed_view.objects[2]

In [ ]:
ed_view.objects[2].object.data[()].data[('HeatMap', 'I')].data

In [ ]:
ed_view.objects[1].object.data[('DynamicMap', 'III')].data[()].data[('Image', 'II')].opts(cmap="kbc_r")

In [ ]:
ed_view.objects[1].object.data[('DynamicMap', 'III')].data[()].data[('Scatter', 'I')].data